# Pru Wealth Data Test

Below we will explore a mock database of customers, policies and funds that are similar to the Pru Wealth data. The first section of this notebook will test your SQL skills, the second section will test your Python skills and visualisation skills.

The database file is called Prudential_Data.db

The three tables included in this database are:
* Customer_Table
* Policy_Table
* Fund_Table

You will need to have the following packages installed in your Python environment:
* numpy
* pandas
* sqlite3

For visualization use any package of your choice (e.g., matplotlib, seaborn, plotly).

Please complete and return the test before the deadline by emailing your copy of this notebook to: Denholm.Hesse@Prudential.co.uk. If you do not know the answer to a question feel free to use Google and Stack Overflow. This is to give you the option to teach yourself a concept from the internet to address a question which you do not know the answer to, please refrain from skipping questions. The test is meant to be completed by you only.

Good luck and have fun!

In [1]:
import numpy as np
import pandas as pd
import sqlite3

# SQL test

IMPORANT: Please make sure all questions in this section are solved using SQL by only editing the content of the variable *sqlite_query*. Please do not edit the below cell:

In [2]:
# Establish DB connection
conn = sqlite3.connect('Prudential_Data.db')

Below is an example of how to query the SQLlite databse from this Jupyter notebook. Please note that SQLlite might have a slightly different syntax from the SQL you normally use. Please do not edit the below cell:

In [3]:
sqlite_query = """
SELECT COUNT(*) 
from policy_table 
"""

pd.read_sql(sqlite_query, conn)

,COUNT(*)
0,1635


The first table "Customer_Table" contains personal information about the customers. Please do not edit the below cell:

In [4]:
sqlite_query = """
SELECT * 
from Customer_Table 
limit 5
"""

pd.read_sql(sqlite_query, conn)

,index,Customer ID,Email,Gender,Age,Postcode
0,0,CO183683,NLUPPITT14@PARALLELS.COM,Male,90,None
1,1,CO314757,None,Female,90,None
2,2,CO720602,None,Female,90,None
3,3,CO34804,None,Female,90,1495-701
4,4,CO474411,None,Male,90,301288


The second table "Policy_Table" contains data relating to customer policies. The policy value is the current value of the policy. Please do not edit the below cell:

In [5]:
sqlite_query = """
SELECT * 
from Policy_Table 
limit 5
"""

pd.read_sql(sqlite_query, conn)

,index,Policy Number,Customer ID,Product Category,Product Name,Policy Value,Policy Start Date
0,0,RET100072,CO276506,RETIREMENT,RETIREMENT ACCOUNT,270033.0,2016-01-02 00:00:00
1,1,RET100931,CO831289,RETIREMENT,RETIREMENT ACCOUNT,26394.0,2016-01-04 00:00:00
2,2,RET100682,CO784249,RETIREMENT,RETIREMENT ACCOUNT,773567.0,2016-01-04 00:00:00
3,3,RET100447,CO295533,RETIREMENT,RETIREMENT ACCOUNT,35512.0,2016-01-07 00:00:00
4,4,RET100390,CO901735,RETIREMENT,RETIREMENT ACCOUNT,767580.0,2016-01-08 00:00:00


The final table "Fund_Table" contains historic data on the Policy Value (Policy_Value_Snapshot). The Policy_Value_Snapshot represents the value of the policy and the Date column is the date at which that value was recorded. The value may have changed overtime due to changes in the fund price or quanity of units the policy was invested in. Please do not edit the below cell:

In [6]:
sqlite_query = """
SELECT 
*
from Fund_Table
limit 5

"""

pd.read_sql(sqlite_query, conn)

,index,Policy Number,Customer ID,Date,Product Category,Product Name,Policy_Value_Snapshot
0,0,ANN1221,CO408564,2009-10-31 00:00:00,ANNUITY,STANDARD ANNUITY,None
1,1,ANN1221,CO408564,2009-11-30 00:00:00,ANNUITY,STANDARD ANNUITY,None
2,2,ANN1221,CO408564,2009-12-31 00:00:00,ANNUITY,STANDARD ANNUITY,None
3,3,ANN1221,CO408564,2010-01-31 00:00:00,ANNUITY,STANDARD ANNUITY,None
4,4,ANN1221,CO408564,2010-02-28 00:00:00,ANNUITY,STANDARD ANNUITY,None


If a question requires additional explanation for your answer, please add additional cells below the code. 

## Q1: What is the current average policy value split by each product name? 

In [7]:
import numpy as np
import pandas as pd
import sqlite3

# Establish DB connection
conn = sqlite3.connect('Prudential_Data.db')

sqlite_query = """
SELECT DISTINCT "Product Name",ROUND(AVG("Policy Value"),0) as "Avg_Policy_Value"
FROM Policy_Table
GROUP BY "Product Name"
ORDER BY "Policy Value" DESC
"""

pd.read_sql(sqlite_query, conn)

,Product Name,Avg_Policy_Value
0,RETIREMENT ACCOUNT,504965.0
1,CORPORATE MEMBERSHIP,5697.0
2,PRUDENTIAL ISA,5161.0
3,FOCUSSED ISA,5746.0
4,PRU BOND,4516.0
5,NHS PENSION,5242.0
6,SIPP,5336.0
7,PRUDENTIAL INVESTMENT PLAN,5085.0
8,STANDARD ANNUITY,NaN


## Q2: How many customers opened a new policy in Q1 2017 who already had an existing pension product? 

In [8]:
import numpy as np
import pandas as pd
import sqlite3

# Establish DB connection
conn = sqlite3.connect('Prudential_Data.db')

sqlite_query = """
SELECT COUNT(*) AS "Cust_Count" FROM
(
    SELECT DISTINCT "Customer ID"
    FROM Policy_Table
    WHERE "Policy Start Date">="2017-04-01 00:00:00" AND "Policy Start Date"<"2017-07-01 00:00:00"
    GROUP BY "Customer ID"
    
INTERSECT

    SELECT DISTINCT "Customer ID"
    FROM Policy_Table
    WHERE "Product Category" = "PENSION" AND "Policy Start Date"<"2017-04-01 00:00:00"
    GROUP BY "Customer ID"
)
"""
pd.read_sql(sqlite_query, conn)

,Cust_Count
0,0


## Q3: What was the customer ID that had the biggest difference in policy value between its earliest and latest date (for any product) ?

In [9]:
import numpy as np
import pandas as pd
import sqlite3

# Establish DB connection
conn = sqlite3.connect('Prudential_Data.db')

sqlite_query = """

SELECT "Customer ID", "Policy Number", MAX(Diff) AS Max_Diff FROM
(
    SELECT  y."Customer ID", y.Date, y."Policy Number", (y.Policy_Value - x.Policy_Value) "Diff"
    FROM
    (
        SELECT  a.* FROM
            (
                SELECT "Customer ID","Policy Number",MAX(Date) AS "Date",ROUND(Policy_Value_Snapshot,0) AS "Policy_Value"
                FROM Fund_Table
                GROUP BY "Customer ID", "Policy Number"
            )
        a INNER JOIN
        (
            SELECT "Customer ID", MAX(Date) MaxDate FROM
            (
                SELECT "Customer ID","Policy Number",MAX(Date) AS "Date",ROUND(Policy_Value_Snapshot,0) AS "Policy_Value"
                FROM Fund_Table
                GROUP BY "Customer ID", "Policy Number"
            )
            GROUP BY "Customer ID"
        
        ) b ON  a."Customer ID" = b."Customer ID" AND a.Date = b.MaxDate
    ) y INNER JOIN
        (
        SELECT  a.* FROM
                (
                SELECT "Customer ID","Policy Number",MIN(Date) AS "Date",ROUND(Policy_Value_Snapshot,0) AS "Policy_Value"
                FROM Fund_Table
                GROUP BY "Customer ID", "Policy Number"
                )
            a INNER JOIN
            (
                SELECT "Customer ID", MIN(Date) MinDate FROM
                (
                    SELECT "Customer ID","Policy Number",MIN(Date) AS "Date",ROUND(Policy_Value_Snapshot,0) AS "Policy_Value"
                    FROM Fund_Table
                    GROUP BY "Customer ID", "Policy Number"
                )
                GROUP BY "Customer ID"
                
            ) b ON  a."Customer ID" = b."Customer ID" AND a.Date = b.MinDate
        ) x ON y."Customer ID" = x."Customer ID"
)

"""

pd.read_sql(sqlite_query, conn)


,Customer ID,Policy Number,Max_Diff
0,CO401070,RET100479,973971.0


# Python Test

Same data, but now let's have some fun in python. Please do not use SQL for this part of the exercise. 

Feel free to add more cells of code or text

In [ ]:
Customer_Table = pd.read_sql('SELECT * FROM Customer_Table', conn)
Policy_Table = pd.read_sql('SELECT * FROM Policy_Table', conn)
Fund_Table = pd.read_sql('SELECT * FROM Fund_Table', conn)

## Q4: Produce a list of customers which have an invalid email address. What do you notice about these customers? ##

Feel free to use any packages of your choice

In [10]:
#Insert your answer below: 

import numpy as np
import pandas as pd
import sqlite3
import re

Customer_Table = pd.read_sql('SELECT * FROM Customer_Table', conn)
Policy_Table = pd.read_sql('SELECT * FROM Policy_Table', conn)
Fund_Table = pd.read_sql('SELECT * FROM Fund_Table', conn)

# Establish DB connection
conn = sqlite3.connect('Prudential_Data.db')

pattern = re.compile(r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)")

df = pd.DataFrame(Customer_Table)
df['Email'].fillna('Missing')
df['Email'] = df['Email'].astype(str)
df['Gender'] = df['Gender'].astype(str)

#print(df.dtypes)

df['isemail'] = df['Email'].apply(lambda x: True if pattern.match(x) else False)
#print(df)

df['isemail'] = df['isemail'].astype(str)
include = ['False']
df_invalid_email = df[df['isemail'].isin(include)] 

print(df_invalid_email)
print(df_invalid_email.describe())

print(df_invalid_email.groupby(df.Gender).describe())

## COMMENTS ##

# There are 53 customer with invalid email address
# Average age for all of these customers is >80

     index Customer ID                     Email  Gender  Age        Postcode  \
1        1    CO314757                      None  Female   90            None   
2        2    CO720602                      None  Female   90            None   
3        3     CO34804                      None  Female   90        1495-701   
4        4    CO474411                      None    Male   90          301288   
5        5    CO777973                      None  Female   90            None   
6        6    CO311505                      None    Male   90            None   
7        7    CO474987                      None  Female   89            None   
8        8    CO496581                      None  Female   89       36520-000   
9        9    CO862381                      None    Male   89            8301   
22      22    CO401070                      None  Female   88            None   
28      28    CO131348                      None    Male   88            None   
29      29    CO494489      

## Q5: The Managing Director of Pru Wealth needs to see how policy value differs between other variables in the database (e.g. customer demographics) . They don't have time to read a report, so have asked for a cool visualisation that tells a story.

Visualise something cool, tell us a story! You can use packages like Matplotlib, Seaborn or Plotly.Tell us why you chose to visualise the data in this way and how it can be used from a business perspective.


In [11]:
# Insert your answer below: 

import numpy as np
import pandas as pd
import sqlite3
import plotly.graph_objects as gp

Customer_Table = pd.read_sql('SELECT * FROM Customer_Table', conn)
Policy_Table = pd.read_sql('SELECT * FROM Policy_Table', conn)
Fund_Table = pd.read_sql('SELECT * FROM Fund_Table', conn)

# Establish DB connection
conn = sqlite3.connect('Prudential_Data.db')

df_policy = pd.DataFrame(Policy_Table)
df_cust = pd.DataFrame(Customer_Table)

# Outer Join to Merge Tables
df = pd.merge(df_policy,df_cust,on='Customer ID',how='outer')

#print(df.describe())

conditions = [
            (df['Age'] < 11), (df['Age'] < 21), (df['Age'] < 31), (df['Age'] < 41),
            (df['Age'] < 51), (df['Age'] < 61), (df['Age'] < 71), (df['Age'] < 81),
            (df['Age'] < 91), (df['Age'] < 101), (df['Age'] > 100)
            ]

categories = ['0-10', '11-20', '21-30','31-40', 
              '41-50', '51-60', '61-70', '71-80',
              '81-90', '91-100', '>100']

df['Age_Cat'] = np.select(condlist=conditions, choicelist=categories)

df = df.groupby(['Age_Cat','Gender']).mean().squeeze().unstack().reset_index()
df = df[['Age_Cat', 'Policy Value']]

df = df.round(decimals=0)

df.columns = ['Age_Category', 'Avg_Policy_Value_Female', 'Avg_Policy_Value_Male']

print (df)
print (df.describe().round(decimals=0))

data = df

y_age = data['Age_Category']
x_M = data['Avg_Policy_Value_Male']
x_F = data['Avg_Policy_Value_Female'] * -1

# Creating instance of the figure
fig = gp.Figure()

# Adding Male data to the figure
fig.add_trace(gp.Bar(y = y_age, x = x_M, name = 'Male', orientation = 'h', marker_color = "rgb(0, 92, 98)"))

# Adding Female data to the figure
fig.add_trace(gp.Bar(y = y_age, x = x_F, name = 'Female', orientation = 'h', marker_color = "rgb(258, 78, 10)"))

# Updating the layout for our graph
fig.update_layout(
    title = 'Average Policy Value Between Various Demographics',
    title_font_size = 22, 
    barmode = 'relative', 
    bargap = 0.2,
    bargroupgap = 0, 
    xaxis = dict(tickvals = [-400000, -300000, -200000, -100000, 0, 100000, 200000, 300000, 400000],
    ticktext = ['400K', '300K', '200K', '100K', '0','100K', '200K', '300K', '400K'],
    title = 'Policy Value in Thousands',
    title_font_size = 14),
    yaxis={'title': 'Age Group'})

fig.show()

## COMMENTS ##

# This gives us both of the demographic variables in single chart to analysis Policy Value
# It is easy to compare average policy value for males at a certain age with the corresponding figures for female
# There is equal distribution of policy value across all ages for males while the same is not true for females.
# Policy Value is lowest for females under 30
# There can be a huge potential among under 30 females in order to get more poicies
# Average policy value for females over 80 is 40% more than males of this age

  Age_Category  Avg_Policy_Value_Female  Avg_Policy_Value_Male
0        11-20                      NaN               337144.0
1        21-30                 234054.0               327515.0
2        31-40                 388100.0               363942.0
3        41-50                 315360.0               307479.0
4        51-60                 346132.0               331668.0
5        61-70                 347252.0               341776.0
6        71-80                 292706.0               361578.0
7        81-90                 393051.0               281456.0
       Avg_Policy_Value_Female  Avg_Policy_Value_Male
count                      7.0                    8.0
mean                  330951.0               331570.0
std                    55847.0                27234.0
min                   234054.0               281456.0
25%                   304033.0               322506.0
50%                   346132.0               334406.0
75%                   367676.0               346726.0
m